In [2]:
import re
from bs4 import BeautifulSoup
import pickle
import pandas as pd
import csv

In [3]:
def store_pickles(filename, to_store):
    """Dumps information in to_store into a file named filename.
    
    filename = string
    to_store = list, dictionary, etc.    
    """  
    with open(filename, 'w') as f:
        pickle.dump(to_store, f)
        
        
def eat_pickles(filename):
    """Loads information from a pickle file.
    
    filename = string
    """
    
    with open(filename, 'r') as f:
        return pickle.load(f)

### Wine.com

In [55]:
wines = eat_pickles('wine_com_dict.pkl')

In [56]:
wine_df = pd.DataFrame(wines)

In [57]:
wine_df.head(25)

,acclaim,name,notes,region,reviews,varietal
0,"[""The 2012 Pinot Noir Russian River Valley imp...",La Crema Russian River Pinot Noir 2012,The 2012 Russian River Valley Pinot Noir opens...,"Russian River, California",[],Pinot Noir
1,"[""Dark ruby. Cherry and dark berries on the n...",Artazu Artazuri Garnacha 2012,A consumer-friendly wine that boasts heady aro...,Spain,[Nice berry on the nose but the promise of a f...,Grenache
2,"[""Lots of oak and lees on this Chardonnay, but...",Dutton Estate Kyndall's Reserve Chardonnay 2012,Kyndall's Reserve Chardonnay is a delightful t...,"Russian River, California",[],Chardonnay
3,"[""Suave and mineral, this wine is blended from...",Chapter 24 Fire + Flood The Fire Pinot Noir 2012,The Fire is all vibrancy and spark which Chapt...,"Willamette Valley, Oregon",[],Pinot Noir
4,N/A,Ghost Pines Chardonnay 2013,Ghost Pines Chardonnay represents the rich win...,California,[],Chardonnay
5,N/A,Sonoma-Cutrer Sonoma Coast Chardonnay 2013,"Fruit aromas of Golden Delicious apple, Bosc p...","Sonoma County, California",[],Chardonnay
6,N/A,Joel Gott Sauvignon Blanc 2013,The 2013 Joel Gott California Sauvignon Blanc ...,California,[Love this wine! Smells like tropical fruits a...,Sauvignon Blanc
7,N/A,Folie a Deux Russian River Chardonnay 2013,"Seductive, complex aromas of honey, green appl...","Russian River, California",[this is a great value at this price!],Chardonnay
8,"[""The 2012 Roberta’s Reserve (100% Merlot aged...",Kapcsandy Family Winery State Lane Vineyard Ro...,N/A,"Napa Valley, California",[],Cabernet Sauvignon
9,N/A,Dom. du Pesquier Cotes du Rhone 2011,"Blend: 56% Grenache, 30% Mourvèdre, 14% Carignan","Cotes du Rhone, Rhone, France",[],Rhone Red Blends


#### Widdle down to wines, not gifts or packages

In [58]:
wine_df = wine_df[wine_df.varietal != "N/A"]

#### Get rid of Vermouth

In [59]:
wine_df = wine_df[wine_df.varietal != 'Vermouth']

In [92]:
# Count how many reviews that are empty
count = 0
for review in wine_df.reviews:
    if len(review) == 0:
        count += 1

In [115]:
# Total number of wines that had reviews in entire data set
print len(df) - count

808


In [113]:
# Count how many acclaims are empty
count3 = 0
for acclaim in wine_df.acclaim:
    if len(acclaim) == 0:
        count3 += 1

In [114]:
# Total number of wines that had acclaims in entire data set
print len(df) - count3

7472


In [189]:
wine_df.varietal.value_counts()

Pinot Noir                    920
Chardonnay                    772
Cabernet Sauvignon            753
Bordeaux Red Blends           535
Other Red Blends              479
Sauvignon Blanc               366
Syrah/Shiraz                  344
Rhone Red Blends              248
Non-Vintage Sparkling Wine    244
Zinfandel                     207
Sangiovese                    196
Merlot                        182
Pinot Gris/Grigio             167
Rosé                          153
Tempranillo                   151
Riesling                      146
Other White Blends            143
Other White Wine              133
Malbec                        133
Other Red Wine                124
Nebbiolo                      110
Rosé Sparkling Wine            94
Other Dessert                  87
Vintage Sparkling Wine         86
Grenache                       81
Port                           62
Rhone White Blends             44
Barbera                        44
Muscat                         42
Viognier      

### Snooth Wine

In [322]:
snooth = eat_pickles('wine_dict_fresh_2.pkl')

In [323]:
snooth_df = pd.DataFrame(snooth)

In [324]:
snooth_df.head()

,color,name,notes,region,reviews,varietal
0,White,Donnafugata Ben Rye 2012,92 points Wine Spectator: A rich and mouthwate...,Italy,[Looking for a sophisticated and refined sweet...,Muscat
1,Red,Jade Mountain Merlot 2012,Jade Mountain showcases the art of blending. W...,USA,[Jade Mountain showcases the art of blending. ...,Merlot
2,Red,W Allegrini Valpol Classico 2008,"A well-structured, complex, elegant and velvet...",Italy,"[Ripe, sweet fruit flavors of damson plum and ...",Valpolicella
3,White,Castello di Pomino Benefizio DOC 2008,The Castello di Pomino was built in the 16th c...,Italy,[One of the major and largest vine growing bus...,Chardonnay
4,Red,Ridge Vineyards Three Valleys Zinfandel 2012,While their Cabernet-based bottling of Monte B...,USA,[Four glasses],Zinfandel


#### Locate non-wines and drop them from the dataframe

In [325]:
non_wine = ['Vodka', 'Vermouth', 'Whiskey', 'Rum', 'Sangria', 'Tequila', 'Liqueur', 'Brandy', 'Scotch', \
        'Single Malt Scotch', 'Scotch Single Malt', 'Bourbon', 'Whiskey Rye', 'Burgundy', 'Irish Whiskey', \
        'Cream Liqueur', 'Spirit' 'Gin', 'Cognac; Armagnac', 'Liqueurs', 'Apertif And Bitters', 'Whiskey Bourbon', \
        'Cognac', 'Whisky', 'Accessories']

indices_to_drop = []

for index, winename in enumerate(snooth_df['name']):
    for nonwine in non_wine:
        if nonwine in winename:
            indices_to_drop.append(index)

In [326]:
indices_to_drop = list(indices_to_drop)

[122, 157, 202, 202, 239, 285, 291, 322, 384, 413, 430, 441, 444, 444, 454, 572, 614, 651, 679, 712, 720, 740, 740, 777, 897, 919, 937, 954, 965, 1064, 1064, 1065, 1106, 1158, 1284, 1286, 1359, 1402, 1456, 1458, 1516, 1540, 1624, 1665, 1772, 1869, 1908, 1926, 1984, 1984, 2107, 2124, 2173, 2208, 2240, 2356, 2423, 2435, 2496, 2508, 2540, 2561, 2623, 2627, 2643, 2646, 2653, 2716, 2762, 2800, 2895, 2899, 2899, 3051, 3076, 3084, 3166, 3197, 3229, 3250, 3323, 3363, 3378, 3383, 3430, 3485, 3489, 3498, 3508, 3520, 3594, 3604, 3650, 3657, 3666, 3679, 3688, 3820, 3878]


In [327]:
indices_to_add = [514, 2659, 2785, 3211, 3297]
indices_to_drop = indices_to_drop + indices_to_add

In [328]:
print indices_to_drop

[122, 157, 202, 202, 239, 285, 291, 322, 384, 413, 430, 441, 444, 444, 454, 572, 614, 651, 679, 712, 720, 740, 740, 777, 897, 919, 937, 954, 965, 1064, 1064, 1065, 1106, 1158, 1284, 1286, 1359, 1402, 1456, 1458, 1516, 1540, 1624, 1665, 1772, 1869, 1908, 1926, 1984, 1984, 2107, 2124, 2173, 2208, 2240, 2356, 2423, 2435, 2496, 2508, 2540, 2561, 2623, 2627, 2643, 2646, 2653, 2716, 2762, 2800, 2895, 2899, 2899, 3051, 3076, 3084, 3166, 3197, 3229, 3250, 3323, 3363, 3378, 3383, 3430, 3485, 3489, 3498, 3508, 3520, 3594, 3604, 3650, 3657, 3666, 3679, 3688, 3820, 3878, 514, 2659, 2785, 3211, 3297]


In [329]:
snooth_df = snooth_df.drop(snooth_df.index[[indices_to_drop]])

In [331]:
snooth_df[snooth_df.varietal.isnull()]

,color,name,notes,region,reviews,varietal
34,White,Jean Lallement Champagne Blend Champagne Brut ...,A vinous style. Bread dough and beef bouillon ...,France,[A vinous style. Bread dough and beef bouillon...,NaN
50,Red,Wine Red Latour 1997,97 points Neal Martin (Wine Advocate): Tasted ...,France,"[Delicious now. Lovely aromas of tobacco, toas...",NaN
54,Rose,J Vineyards & Winery Brut Rosé 2004,Type of Vine: Meunier\n\nAlcohol Percentage: 12 %,USA,[Five glasses],NaN
60,White,Westmalle Trappist Ale Tripel .2 Oz 2011,"Westmalle Tripel is a clear, golden yellow Tra...",Belgium,"[Westmalle Tripel is a clear, golden yellow Tr...",NaN
64,Red,Domaine de la Mordorée Red Rhone Blend Château...,"This is an impressive wine in which Syrah, Gre...",France,[Luxurious Drinks offers you: Wine - red - 200...,NaN
65,Red,Dios Baco Baco Fino,"Aromas of fresh cut straw, almonds and sweet c...",Spain,[],NaN
67,White,Yoshinogawa Gensen Karakuchi NV,Yoshinogawa Gensen Karakuchi Sake 720ML The na...,Japan,[],NaN
68,White,Jean Laurent Brut Rosé Champagne NV,"A deep-hued rosé, almost garnet in color, with...",France,"[A deep-hued rosé, almost garnet in color, wit...",NaN
77,Red,Pierre Henri Morel Gigondas 2010,"""Ruby-red. Ripe dark berry and cherry aromas a...",France,[Inky and big alcohol on the nose. Very tight...,NaN
85,Red,Celler El Masroig Castell Pinyeres 2009,"Montsant, SpainCeller el Masroig Castell de le...",Spain,"[Montsant, SpainCeller el Masroig Castell de l...",NaN


In [332]:
count2 = 0
for review in snooth_df.reviews:
    if len(review) == 0:
        count2 += 1

In [333]:
print len(snooth_df) - count2

3126


In [334]:
snooth_df.varietal.value_counts()

Chardonnay                    480
Cabernet Sauvignon            413
Pinot Noir                    339
Syrah                         184
Other Red Blends              171
Sauvignon Blanc               145
Merlot                        136
Sangiovese                    126
Riesling                       99
Zinfandel                      98
Nebbiolo                       84
Malbec                         71
Pinot Grigio                   70
Other White Blends             70
Muscat                         65
Other White Wine               57
Other Red Wine                 54
Grenache                       50
Tempranillo                    50
Rhone Red Blends               44
Non-Vintage Sparkling Wine     42
Bordeaux Red Blends            36
Research                       34
Sherry                         26
Valpolicella                   23
Barbera                        21
Cabernet Franc                 19
Chenin Blanc                   16
Mourvedre                      14
Pinot Blanc   

In [127]:
snooth_df.ix[287]['reviews']

[u'I came across this wine last year. And this year it tastes even better! Great with fish, chicken and excellant with salads.  Nice looking bottle to give as a hostess gift. Friends seem to enjoy it too.',
 u'Darker shad of gold that resembles a Chardonnay, which indeed is what it states it is mostly on the bottle. Marketed as a table white. Very aromatic with vanilla and maple syrup on the nose. On the extreme side of sweetness compared to others in the Chardonnay class and resembles a palate more similar to a Riesling. Tastes somewhat like the nose with vanilla but a hint of a buttery flavor not unlike a typical Chardonnay. Overall, I would say not a bad wine for CT, but definitely not your typical Chard.',
 u'Though a touch too sweet for me if I have more than a couple of glasses, this is a refreshing wine, great for the summer months...Beautiful vineyard too...']